In [24]:
# Install properly versioned dependencies
%pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Note: you may need to restart the kernel to use updated packages.


In [25]:
import os
import sys

# Patch the path to include our libs
sys.path.insert(0, os.path.abspath('./libs'))

In [ ]:
from ipywidgets import Button

In [26]:

# Neural Network Deps

import pytorch_lightning as pl
from data.data_loader import MyDataModule
from training.PL_train import Main_Loop
import torch.optim as optim
import torchio as tio
import torch
import numpy as np

In [27]:
size = (48, 64, 48)
model = "Pixel"
criterion = "Tversky"
type_list = ["t1"]
prepare = False
epochs = 100
weight = torch.from_numpy(np.array([.01,2,2,2,2])).float().cuda()
model_args = {
            }
# BRATS Dataset
data_dir = os.path.abspath('./data/raw')

# Pre processings location
out_dir = os.path.abspath('./data/processed')


# Data TRansforms
train_transformer = tio.Compose([
    tio.RandomMotion(p=0.2),
    tio.RandomBiasField(p=0.3),
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),
    tio.RandomNoise(p=0.5),
    tio.RandomFlip(),
    tio.OneOf({
        tio.RandomAffine(): 0.8,
        tio.RandomElasticDeformation(): 0.2,
    }),
])

val_transformer = tio.Compose([
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),
])


# Dataloading
data_module = MyDataModule(
    data_dir = data_dir,
    out_dir = out_dir,
    train_transformer=train_transformer,
    val_transformer = val_transformer,
    prepare=prepare,
    size = size,
    type_list = type_list,
    sample_list= type_list
)

In [28]:

trainer = pl.Trainer(gpus=1,max_epochs = epochs)
main = Main_Loop(
    model = model,
    loss= criterion,
    type_list = type_list,
    scheduler = optim.lr_scheduler.ExponentialLR,
    scheduler_args = {"gamma":.96},
    model_args = model_args,
    loss_args={"weight":weight  ,"alpha":.5 }
)
trainer.fit(main, data_module)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/josh/Repos/tumor-segmentation/lightning_logs


/home/josh/Repos/tumor-segmentation/data/processed


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.